In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install resemblyzer

In [ ]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = '/content/drive/MyDrive/X2zqiX6yL3I.mp3'
wav_fpath = Path("/content/drive/MyDrive/X2zqiX6yL3I.mp3")

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


Loaded the voice encoder model on cpu in 0.01 seconds.
(2055, 256)


In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment

def mp3_to_wav(audio_file_path):
    sound = AudioSegment.from_mp3(audio_file_path)
    audio_file_path = audio_file_path.split('.')[0] + '.wav'
    sound.export(audio_file_path, format="wav")
    return audio_file_path

audio_file_path = mp3_to_wav(audio_file_path)
print(audio_file_path)

/content/drive/MyDrive/X2zqiX6yL3I.wav


In [ ]:
!pip install spectralcluster

In [ ]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=2,
    max_clusters=100,
    p_percentile=0.90,
    gaussian_blur_sigma=1)

labels = clusterer.predict(cont_embeds)
print(labels)

[0 0 0 ... 0 0 0]


In [ ]:
def create_labelling(labels,wav_splits):
    from resemblyzer.audio import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)
print(labelling)

[('0', 0, 6.56), ('1', 6.56, 12.32), ('0', 12.32, 40.04), ('1', 40.04, 41.78), ('0', 41.78, 42.32), ('1', 42.32, 42.44), ('0', 42.44, 43.88), ('1', 43.88, 44.0), ('0', 44.0, 56.84), ('1', 56.84, 71.72), ('0', 71.72, 124.04)]


In [ ]:
!pip install ffmpeg-python


In [ ]:
import ffmpeg

In [ ]:
for ele in labelling:
  start=ele[1]
  end=ele[2]
  stream = ffmpeg.input(audio_file_path)
  stream=ffmpeg.trim(stream,start = start, end = end)
  stream = ffmpeg.output(stream,'/content/drive/MyDrive/output.mp3')
  ffmpeg.run(stream)


 

  


Error: ignored

In [ ]:
!pip install SpeechRecognition pydub

     |████████████████████████████████| 32.8MB 87kB/s 


In [ ]:
import speech_recognition as sr

In [ ]:
r = sr.Recognizer()
with sr.AudioFile(test) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print(text)

FileNotFoundError: ignored